In [1]:
import numpy as np
import pandas as pd
import cypari2

from tqdm.notebook import tqdm

In [2]:
pari = cypari2.Pari()
pari.allocatemem(2**32)

PARI stack size set to 4294967296 bytes, maximum size set to 4294967296


In [3]:
pari.variable()

[x, y]

# Find max dimension of the coeff vectors

In [4]:
def min_max_degree_J(list_of_poly, variables=['q']):
    min_max = {}
    for v in variables:
        min_max[v] = {}
        min_max[v]['max'] = -1000
        min_max[v]['min'] = +1000
        
    for poly in list_of_poly:
        poly_J = pari.subst(poly, 't', -1) / pari('q + q^-1')
        
        for v in variables:
            # max degree of the Jones poly, note the division by two!!
            for i in range(100, -10, -1):
                i_degree = pari.polcoef(pari(poly_J), i, v)
                if i_degree != 0:
                    if i / 2 > min_max[v]['max']:
                        min_max[v]['max'] = int(i/2)
                    break
                
            # min degree
            # goes from -100 to 10 and stops
            # at the first non zero coeff
            for i in range(-100, 10):
                i_degree = pari.polcoef(pari(poly_J), i, v)
                if i_degree != 0:
                    if i / 2 < min_max[v]['min']:
                        min_max[v]['min'] = int(i/2)
                    break
    
    return min_max

# Create vector of coeff

In [5]:
def create_coeff_vector(poly, max_q, min_q):
    off_q = -min_q

    coeff = np.zeros(max_q-min_q+1, dtype=int)

    poly_J = pari.subst(poly, 't', -1) / pari('q + q^-1')

    for idx_q in range(min_q, max_q+1):
        coeff[idx_q+off_q] = pari.polcoef(poly_J, idx_q*2, 'q')
            
    return coeff

def create_colnames(max_q, min_q):
    colnames = []
    for idx_q in range(min_q, max_q+1):
        colnames.append('q'+str(idx_q))
    return colnames

# all up to 13

In [9]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_data")

Loading Khovanov polynomials for knots with up to 13 crossings ... done.


In [10]:
poly_vectors = [
'KhPolQ_K3a',
'KhPolQ_K4a',
'KhPolQ_K5a',
'KhPolQ_K6a',
'KhPolQ_K7a',
'KhPolQ_K8a',
'KhPolQ_K8n',
'KhPolQ_K9a', 
'KhPolQ_K9n',
'KhPolQ_K10a',
'KhPolQ_K10n',
'KhPolQ_K11a',
'KhPolQ_K11n',
'KhPolQ_K12a',
'KhPolQ_K12n',
'KhPolQ_K13a',
'KhPolQ_K13n']


In [11]:
min_max_list = []

for poly_vec in poly_vectors:
    for i in tqdm(range(0, len(pari(poly_vec)), 1000)):
        #print(i, i+1000)
        min_max_list.append(min_max_degree_J(pari(poly_vec)[i:i+1000]))

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])

size_q = max(abs(max_q), abs(min_q)) 

print('q', min_q, max_q)
print(size_q)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

q -15 19
19


In [34]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q)
        c_list.append(p_coeff)
        c_list.append(p_coeff[::-1])

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

KhPolQ_K3a


|          | 0/? [00:00<?, ?it/s]

KhPolQ_K4a


|          | 0/? [00:00<?, ?it/s]

KhPolQ_K5a


...

|          | 0/? [00:00<?, ?it/s]

KhPolQ_K13a


|          | 0/? [00:00<?, ?it/s]

KhPolQ_K13n


|          | 0/? [00:00<?, ?it/s]

In [13]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q))
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

In [15]:
coeff_df

,knot_id,q-19,q-18,q-17,q-16,q-15,q-14,q-13,q-12,q-11,...,q12,q13,q14,q15,q16,q17,q18,q19,is_alternating,number_of_crossings
0,KhPolQ_K3a_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
1,KhPolQ_K3a_1!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
2,KhPolQ_K4a_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
3,KhPolQ_K4a_1!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
4,KhPolQ_K5a_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25925,KhPolQ_K13n_5108!,0,0,0,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,13
25926,KhPolQ_K13n_5109,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13
25927,KhPolQ_K13n_5109!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13
25928,KhPolQ_K13n_5110,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13


In [18]:
## Add s-invariant
#pari.read("raw/Khovanov+Rasmussen/s-value_knots")
#s_list = [int(s) for s in pari('s_value_K13n')]
#s_list_double = []
#for s in s_list:
#    s_list_double.append(s)
#    s_list_double.append(-s)
coeff_df['s-value'] = pd.NA

In [25]:
signature_files = [
'03a',
'04a',
'05a',
'06a',
'07a',
'08a',
'08n',
'09a', 
'09n',
'10a',
'10n',
'11a',
'11n',
'12a',
'12n',
'13a',
'13n']

In [30]:
## Add signature
sign_list = []

for k in signature_files:
    signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-{}-Signature'.format(k), 
                              header=None, names = ['na','cross','mirror','signature'])
    signature_df.reset_index(inplace=True)
    sign_list += signature_df.signature.values.tolist()
    
coeff_df['signature'] = sign_list

In [31]:
coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [32]:
coeff_df

,knot_id,number_of_crossings,is_alternating,signature,s-value,q-19,q-18,q-17,q-16,q-15,...,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19
0,KhPolQ_K3a_1,3,1,-2,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,KhPolQ_K3a_1!,3,1,2,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KhPolQ_K4a_1,4,1,0,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KhPolQ_K4a_1!,4,1,0,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KhPolQ_K5a_1,5,1,-2,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25925,KhPolQ_K13n_5108!,13,0,4,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25926,KhPolQ_K13n_5109,13,0,-2,<NA>,0,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
25927,KhPolQ_K13n_5109!,13,0,2,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25928,KhPolQ_K13n_5110,13,0,-2,<NA>,0,0,0,0,0,...,-2,0,0,0,0,0,0,0,0,0


In [33]:
coeff_df.to_csv('parsed/Khovanov/Jones_UP_TO_13_MIRROR.csv', header=True, index=False)

# 12a

In [ ]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_data")

In [ ]:
poly_vectors = ['KhPolQ_K12a']
min_max_list = [min_max_degree_J(pari(vec)) for vec in poly_vectors]

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])

size_q = max(abs(max_q), abs(min_q)) 

print('q', min_q, max_q)
print(size_q)

In [ ]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q)
        c_list.append(p_coeff)
        c_list.append(p_coeff[::-1])

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')
        

In [ ]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q))
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

In [ ]:
## Add s-invariant
pari.read("raw/Khovanov+Rasmussen/s-value_knots")
s_list = [int(s) for s in pari('s_value_K12a')]
s_list_double = []
for s in s_list:
    s_list_double.append(s)
    s_list_double.append(-s)
    
coeff_df['s-value'] = s_list_double

In [ ]:
## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-12a-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
#signature_df = signature_df[~(signature_df.mirror.str[-1] == '!')]
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

In [ ]:
coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [ ]:
coeff_df.loc[2574].values

In [ ]:
coeff_df.loc[2575].values

In [ ]:
coeff_df.to_csv('parsed/Khovanov/Jones_12a_MIRROR.csv', header=True, index=False)

# 12n

In [ ]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_data")

In [ ]:
poly_vectors = ['KhPolQ_K12n']
min_max_list = [min_max_degree_J(pari(vec)) for vec in poly_vectors]

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])

size_q = max(abs(max_q), abs(min_q)) 

print('q', min_q, max_q)
print(size_q)

In [ ]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q)
        c_list.append(p_coeff)
        c_list.append(p_coeff[::-1])

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

In [ ]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q))
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

In [ ]:
## Add s-invariant
pari.read("raw/Khovanov+Rasmussen/s-value_knots")
s_list = [int(s) for s in pari('s_value_K12n')]
s_list_double = []
for s in s_list:
    s_list_double.append(s)
    s_list_double.append(-s)
coeff_df['s-value'] = s_list_double

In [ ]:
## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-12n-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

In [ ]:
coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [ ]:
coeff_df

In [ ]:
coeff_df.loc[1774].values

In [ ]:
coeff_df.loc[1775].values

In [ ]:
coeff_df.to_csv('parsed/Khovanov/Jones_12n_MIRROR.csv', header=True, index=False)

# 13a

In [ ]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_data")

In [ ]:
poly_vectors = ['KhPolQ_K13a']
min_max_list = []

min_max_list.append(min_max_degree_J(pari('KhPolQ_K13a')[:1000]))
min_max_list.append(min_max_degree_J(pari('KhPolQ_K13a')[1000:2010]))
min_max_list.append(min_max_degree_J(pari('KhPolQ_K13a')[2000:3010]))
min_max_list.append(min_max_degree_J(pari('KhPolQ_K13a')[3000:4010]))
min_max_list.append(min_max_degree_J(pari('KhPolQ_K13a')[4000:]))

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])

size_q = max(abs(max_q), abs(min_q)) 

print('q', min_q, max_q)
print(size_q)

In [ ]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q)
        c_list.append(p_coeff)
        c_list.append(p_coeff[::-1])

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

In [ ]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q))
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

In [ ]:
coeff_df

In [ ]:
## Add s-invariant
pari.read("raw/Khovanov+Rasmussen/s-value_knots")
s_list = [int(s) for s in pari('s_value_K13a')]
s_list_double = []
for s in s_list:
    s_list_double.append(s)
    s_list_double.append(-s)
coeff_df['s-value'] = s_list_double

In [ ]:
## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-13a-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

In [ ]:
coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [ ]:
coeff_df

In [ ]:
coeff_df.to_csv('parsed/Khovanov/Jones_13a_MIRROR.csv', header=True, index=False)

# 13n

In [ ]:
poly_vectors = ['KhPolQ_K13n']
min_max_list = []

min_max_list.append(min_max_degree_J(pari('KhPolQ_K13n')[:1000]))
min_max_list.append(min_max_degree_J(pari('KhPolQ_K13n')[1000:2010]))
min_max_list.append(min_max_degree_J(pari('KhPolQ_K13n')[2000:3010]))
min_max_list.append(min_max_degree_J(pari('KhPolQ_K13n')[3000:4010]))
min_max_list.append(min_max_degree_J(pari('KhPolQ_K13n')[4000:]))

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])

size_q = max(abs(max_q), abs(min_q)) 

print('q', min_q, max_q)
print(size_q)

In [ ]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q)
        c_list.append(p_coeff)
        c_list.append(p_coeff[::-1])

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

In [ ]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q))
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

In [ ]:
## Add s-invariant
pari.read("raw/Khovanov+Rasmussen/s-value_knots")
s_list = [int(s) for s in pari('s_value_K13n')]
s_list_double = []
for s in s_list:
    s_list_double.append(s)
    s_list_double.append(-s)
coeff_df['s-value'] = s_list_double

In [ ]:
## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-13n-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

In [ ]:
coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [ ]:
coeff_df

In [ ]:
coeff_df.to_csv('parsed/Khovanov/Jones_13n_MIRROR.csv', header=True, index=False)

# 14n

In [6]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_14n_data")

Loading Khovanov polynomials for non-alternating knots with 14 crossings ... done.


In [7]:
poly_vectors = ['KhPolQ_K14n']
min_max_list = []

for i in tqdm(range(0, len(pari('KhPolQ_K14n')), 1000)):
    #print(i, i+1000)
    min_max_list.append(min_max_degree_J(pari('KhPolQ_K14n')[i:i+1000]))

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])

size_q = max(abs(max_q), abs(min_q)) 

print('q', min_q, max_q)
print(size_q)

0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000

q -17 19
19


In [8]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q)
        c_list.append(p_coeff)
        c_list.append(p_coeff[::-1])

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

KhPolQ_K14n


In [9]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q))
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

## Add s-invariant
pari.read("raw/Khovanov+Rasmussen/s-value_14n")
s_list = [int(s) for s in pari('s_value_K14n')]
s_list_double = []
for s in s_list:
    s_list_double.append(s)
    s_list_double.append(-s)
coeff_df['s-value'] = s_list_double

## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-14n-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [10]:
coeff_df

,knot_id,number_of_crossings,is_alternating,signature,s-value,q-19,q-18,q-17,q-16,q-15,...,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19
0,KhPolQ_K14n_1,14,0,2,-2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,KhPolQ_K14n_1!,14,0,-2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KhPolQ_K14n_2,14,0,-4,4,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,KhPolQ_K14n_2!,14,0,4,-4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KhPolQ_K14n_3,14,0,2,-2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54867,KhPolQ_K14n_27434!,14,0,10,-12,1,-4,7,-11,14,...,0,0,0,0,0,0,0,0,0,0
54868,KhPolQ_K14n_27435,14,0,-8,10,0,0,0,0,0,...,8,-13,16,-17,16,-12,7,-4,1,0
54869,KhPolQ_K14n_27435!,14,0,8,-10,0,1,-4,7,-12,...,0,0,0,0,0,0,0,0,0,0
54870,KhPolQ_K14n_27436,14,0,-8,10,0,0,0,0,0,...,14,-21,27,-27,24,-19,11,-5,1,0


In [11]:
coeff_df.to_csv('parsed/Khovanov/Jones_14n_MIRROR.csv', header=True, index=False)

# 15n

In [6]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_15n_data")

Loading Khovanov polynomials for non-alternating knots with 15 crossings ... done.


In [7]:
poly_vectors = ['KhPolQ_K15n']
min_max_list = []

for i in tqdm(range(0, len(pari('KhPolQ_K15n')), 1000)):
    #print(i, i+1000)
    min_max_list.append(min_max_degree_J(pari('KhPolQ_K15n')[i:i+1000]))

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])

size_q = max(abs(max_q), abs(min_q)) 

print('q', min_q, max_q)
print(size_q)

  0%|          | 0/169 [00:00<?, ?it/s]

q -18 20
20


In [8]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q)
        c_list.append(p_coeff)
        c_list.append(p_coeff[::-1])

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

KhPolQ_K15n


0it [00:00, ?it/s]

In [9]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q))
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

## Add s-invariant
pari.read("raw/Khovanov+Rasmussen/s-value_15n")
s_list = [int(s) for s in pari('s_value_K15n')]
s_list_double = []
for s in s_list:
    s_list_double.append(s)
    s_list_double.append(-s)
coeff_df['s-value'] = s_list_double

## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-15n-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

In [11]:
coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [12]:
coeff_df

,knot_id,number_of_crossings,is_alternating,signature,s-value,q-20,q-19,q-18,q-17,q-16,...,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20
0,KhPolQ_K15n_1,15,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,KhPolQ_K15n_1!,15,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KhPolQ_K15n_2,15,0,4,-2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KhPolQ_K15n_2!,15,0,-4,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KhPolQ_K15n_3,15,0,-2,4,0,0,0,0,0,...,3,-1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336055,KhPolQ_K15n_168028!,15,0,6,-8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336056,KhPolQ_K15n_168029,15,0,-2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336057,KhPolQ_K15n_168029!,15,0,2,-2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336058,KhPolQ_K15n_168030,15,0,-4,4,0,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0


In [13]:
coeff_df.to_csv('parsed/Khovanov/Jones_15n_MIRRORS.csv', header=True, index=False)

# 16n

In [9]:
%%time
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_16n_data")

Loading Khovanov polynomials for non-alternating knots with 16 crossings ... done.
CPU times: user 24min 24s, sys: 22.4 s, total: 24min 47s
Wall time: 27min 50s


In [7]:
poly_vectors = ['KhPolQ_K16n']
min_max_list = []

for i in tqdm(range(0, len(pari('KhPolQ_K16n')), 1000)):
    #print(i, i+1000)
    min_max_list.append(min_max_degree_J(pari('KhPolQ_K16n')[i:i+1000]))

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])

size_q = max(abs(max_q), abs(min_q)) 

print('q', min_q, max_q)
print(size_q)

  0%|          | 0/1009 [00:00<?, ?it/s]

q -20 22
22


In [8]:
max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])

size_q = max(abs(max_q), abs(min_q)) 

print('q', min_q, max_q)
print(size_q)

q -20 22
22


In [6]:
size_q = 22

In [9]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q)
        c_list.append(p_coeff)
        c_list.append(p_coeff[::-1])

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

KhPolQ_K16n


|          | 0/? [00:00<?, ?it/s]

In [11]:
with open('c_list.npy', 'wb') as f:
    np.save(f, c_list)
    
with open('knot_id_list.npy', 'wb') as k:
    np.save(k, knot_id_list)

In [13]:
c_list[0].shape

(45,)

In [7]:
with open('c_list.npy', 'rb') as f:
    c_list = np.load(f)

In [8]:
len(c_list)

2017812

In [10]:
knot_id_list = []
poly_vectors = ['KhPolQ_K16n']

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

KhPolQ_K16n


|          | 0/? [00:00<?, ?it/s]

In [11]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q))
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

## Add s-invariant
## not present for 16n
# pari.read("raw/Khovanov+Rasmussen/s-value_16n")
# s_list = [int(s) for s in pari('s_value_K16n')]
# s_list_double = []
# for s in s_list:
#     s_list_double.append(s)
#     s_list_double.append(-s)
# coeff_df['s-value'] = s_list_double
#coeff_df['s-value'] = pd.NA

## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-16n-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

In [14]:
coeff_df

,knot_id,q-22,q-21,q-20,q-19,q-18,q-17,q-16,q-15,q-14,...,q16,q17,q18,q19,q20,q21,q22,is_alternating,number_of_crossings,signature
0,KhPolQ_K16n_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16,0
1,KhPolQ_K16n_1!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16,0
2,KhPolQ_K16n_2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16,0
3,KhPolQ_K16n_2!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16,0
4,KhPolQ_K16n_3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017807,KhPolQ_K16n_1008904!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16,6
2017808,KhPolQ_K16n_1008905,0,0,0,0,0,0,0,0,0,...,55,-36,17,-6,1,0,0,0,16,-8
2017809,KhPolQ_K16n_1008905!,0,0,1,-6,17,-36,55,-73,84,...,0,0,0,0,0,0,0,0,16,8
2017810,KhPolQ_K16n_1008906,0,0,0,0,0,0,0,0,0,...,55,-36,17,-6,1,0,0,0,16,-8


In [15]:
coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature'] 
                     + coeff_df.columns.to_list()[1:-3]]

In [16]:
coeff_df.head()

,knot_id,number_of_crossings,is_alternating,signature,q-22,q-21,q-20,q-19,q-18,q-17,...,q13,q14,q15,q16,q17,q18,q19,q20,q21,q22
0,KhPolQ_K16n_1,16,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,KhPolQ_K16n_1!,16,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KhPolQ_K16n_2,16,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KhPolQ_K16n_2!,16,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KhPolQ_K16n_3,16,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
coeff_df.to_csv('parsed/Khovanov/Jones_16n_MIRRORS.csv', header=True, index=False)

# merge files

In [1]:
import pandas as pd

## 14

In [2]:
%%time
jones_13 = pd.read_csv('parsed/Khovanov/Jones_UP_TO_13_MIRROR.csv')
jones_14n = pd.read_csv('parsed/Khovanov/Jones_14n_MIRROR.csv')

CPU times: user 557 ms, sys: 159 ms, total: 715 ms
Wall time: 857 ms


In [3]:
jones_13n = jones_13[jones_13.is_alternating == 0].copy()

In [4]:
print(jones_13n.shape)
print(jones_14n.shape)

(12472, 44)
(54872, 44)


In [6]:
missing_cols = [c for c in jones_14n.columns if c not in jones_13n.columns]
jones_13n[missing_cols] = 0

In [7]:
%%time
jones_upto_14n = pd.concat([jones_13n, jones_14n], ignore_index=True)

CPU times: user 23.2 ms, sys: 18.2 ms, total: 41.4 ms
Wall time: 54 ms


## 15

In [11]:
%%time
jones_15n = pd.read_csv('parsed/Khovanov/Jones_15n_MIRRORS.csv')

In [12]:
missing_cols = [c for c in jones_15n.columns if c not in jones_upto_14n.columns]
jones_upto_14n[missing_cols] = 0

In [13]:
print(jones_upto_14n.shape)
print(jones_15n.shape)

(67344, 46)
(336060, 46)


In [14]:
%%time
jones_upto_15n = pd.concat([jones_upto_14n, jones_15n], ignore_index=True)

CPU times: user 292 ms, sys: 224 ms, total: 516 ms
Wall time: 602 ms


## 16

In [15]:
%%time
jones_16n = pd.read_csv('parsed/Khovanov/Jones_16n_MIRRORS.csv')

CPU times: user 16.1 s, sys: 2.74 s, total: 18.8 s
Wall time: 22.3 s


In [16]:
missing_cols = [c for c in jones_16n.columns if c not in jones_upto_15n.columns]
jones_upto_15n[missing_cols] = 0

In [17]:
print(jones_upto_15n.shape)
print(jones_16n.shape)

(403404, 50)
(2017812, 49)


In [20]:
%%time
jones_upto_16n = pd.concat([jones_upto_15n, jones_16n], ignore_index=True)

CPU times: user 1.89 s, sys: 1.59 s, total: 3.48 s
Wall time: 3.78 s


In [22]:
jones_upto_16n.drop(columns=['s-value'], inplace=True)

In [23]:
jones_upto_16n

,knot_id,number_of_crossings,is_alternating,signature,q-19,q-18,q-17,q-16,q-15,q-14,...,q16,q17,q18,q19,q-20,q20,q-22,q-21,q21,q22
0,KhPolQ_K8n_1,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,KhPolQ_K8n_1!,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KhPolQ_K8n_2,8,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KhPolQ_K8n_2!,8,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KhPolQ_K8n_3,8,0,-6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2421211,KhPolQ_K16n_1008904!,16,0,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2421212,KhPolQ_K16n_1008905,16,0,-8,0,0,0,0,0,0,...,55,-36,17,-6,0,1,0,0,0,0
2421213,KhPolQ_K16n_1008905!,16,0,8,-6,17,-36,55,-73,84,...,0,0,0,0,1,0,0,0,0,0
2421214,KhPolQ_K16n_1008906,16,0,-8,0,0,0,0,0,0,...,55,-36,17,-6,0,1,0,0,0,0


In [27]:
%%time
jones_upto_16n.to_csv('parsed/Jones_fromKhov_upto_16n_MIRRORS.csv', header=True, index=False)

CPU times: user 43.2 s, sys: 1.55 s, total: 44.7 s
Wall time: 51.8 s


In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('parsed/Jones_fromKhov_upto_16n_MIRRORS.csv')

In [3]:
data[data.number_of_crossings < 16]

,knot_id,number_of_crossings,is_alternating,signature,q-19,q-18,q-17,q-16,q-15,q-14,...,q16,q17,q18,q19,q-20,q20,q-22,q-21,q21,q22
0,KhPolQ_K8n_1,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,KhPolQ_K8n_1!,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KhPolQ_K8n_2,8,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KhPolQ_K8n_2!,8,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KhPolQ_K8n_3,8,0,-6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403399,KhPolQ_K15n_168028!,15,0,6,0,0,0,0,1,-5,...,0,0,0,0,0,0,0,0,0,0
403400,KhPolQ_K15n_168029,15,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
403401,KhPolQ_K15n_168029!,15,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
403402,KhPolQ_K15n_168030,15,0,-4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data[data.number_of_crossings < 16].to_csv('parsed/Jones_fromKhov_upto_15n_MIRRORS.csv', header=True, index=False)